In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Sat Jun 29 16:47:38 PDT 2024


# Scenario 1, 2, 3: non-TDA approaches to DAS

In [2]:
np.random.seed(12345)  # set random seed for reproducibility

# Load synthetic data for TX and use it to simulate 2010 and 2020 populations

In [3]:
import linked_census_disclosure.data as lcd_data

In [4]:
%%time

df = lcd_data.read_synth_data('tx')

CPU times: user 40.1 s, sys: 8.25 s, total: 48.3 s
Wall time: 50.7 s


In [5]:
# does this have the expected number of rows?
f'{len(df):,.0f}' # expect population of texas in 2010 to be 25,145,561

'25,145,561'

# Focus in on the 10-17 year olds

We will use this group as our 2020 population and then simulate what their attributes were in 2010 for our 2010 population.

In [6]:
df_all = df
df = df[(df.age >= 10) & (df.age < 18)].copy()

n_kids = len(df)  # number of children
f'{n_kids:,.0f}'

'3,009,117'

In [7]:
#### simulate 10 years of demographic change

df['age_2020'] = df.age
df['age_2010'] = df.age - 10
del df['age']

In [8]:
assert np.all(df.age_2010 >= 0), 'ensure that all ages are still non-negative'

In [9]:
# ignore births, because we are focused
# only on kids who can be linked between 2010 and 2020 census

# Add in migration, make the change to the 2010 geography

p_stay from ACS, see [2022_04_19a_das_dhc_attack_mig_data.ipynb](2022_04_19a_das_dhc_attack_mig_data.ipynb)

In [10]:
# simple model migration, based on probability
# of being in same house for 10+ years among household with 8-17 year olds
# in ACS

p_stay = 0.23


In [11]:
all_locations = df_all.hh_id.unique()

In [12]:
locations_2020 = df.hh_id.unique()

In [13]:
random_location = np.random.choice(all_locations, size=len(locations_2020),
                     replace=True)

locations_2010 = np.where(np.random.uniform(size=len(locations_2020)) < p_stay,
                         locations_2020,
                         random_location)

s_location_2010 = pd.Series(locations_2010,
                            index=locations_2020)

In [14]:
np.mean(locations_2010 == locations_2020)  # should be around 0.23

0.22957601574075331

In [15]:
df['hh_id_2020'] = df.hh_id
df['hh_id_2010'] = df.hh_id.map(s_location_2010)
del df['hh_id']

df

,state,county,tract,block,geoid,sex_id,relationship,hispanic,racaian,racasn,racblk,racnhpi,racsor,racwht,pweight,age_2020,age_2010,hh_id_2020,hh_id_2010
2,48,61,980100,1000,480619801001000,1,25,1,0,0,0,0,0,1,1,15,5,480619801001000-1,483090037102003-1
6,48,61,980100,1000,480619801001000,2,25,1,0,0,0,0,0,1,1,10,0,480619801001000-2,484790012011000-16
7,48,61,980100,1000,480619801001000,2,25,1,0,0,0,0,0,1,1,15,5,480619801001000-2,484790012011000-16
16,48,441,11600,1000,484410116001000,1,20,0,0,0,0,0,0,1,1,13,3,484410116001000-52,484410116001000-52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25145326,48,113,7916,2001,481130079162001,2,22,0,0,1,0,0,0,0,1,17,7,481130079162001-743,483396940011002-29
25145453,48,113,7916,1002,481130079161002,2,28,1,0,0,0,0,0,1,1,14,4,481130079161002-738,484910215152000-16
25145494,48,113,7916,2001,481130079162001,1,25,1,0,0,0,0,1,0,1,10,0,481130079162001-788,484910201171011-2
25145495,48,113,7916,2001,481130079162001,1,36,1,0,0,0,0,1,0,1,10,0,481130079162001-791,481410041031009-356


In [16]:
np.mean(df.hh_id_2010 == df.hh_id_2020)  # should be around 23%

0.22982755406320193

In [17]:
df['geoid_2020'] = df.geoid
df['geoid_2010'] = df.hh_id_2010.map(lambda x: int(x.split('-')[0]))

In [18]:
np.mean(df.geoid_2010 == df.geoid_2020)  # should be around 23%

0.22983353588444716

# Make gender column, based on BRFSS 2019 SOGI results

In future work, could try to incorporate observation that there is substantial age dependence in these values.  But for now, keep it simple, and work with the crude prevalence rates.

In [19]:
df['reported_sex_2020'] = df.sex_id

In [20]:
# then initalize a gender for each simulant, calibrated to have unconditional probability from BRFSS
# and also to match the data generation procedure for the reported_sex_2020 column

p_trans_boy = 0.18 / 100
p_trans_girl = 0.22 / 100
p_trans_other = 0.12 / 100
p_cis = 98.08 / 100

# rescale to sum to 100%
p_gender = np.array([p_trans_boy, p_trans_girl, p_trans_other, p_cis])
p_gender /= p_gender.sum()
p_gender

array([0.00182556, 0.00223124, 0.00121704, 0.99472617])

In [21]:
# first initialize gender without distinguishing cis boy and cis girl
# since that matches BRFSS SOGI question
df['gender'] = np.random.choice(['trans_boy', 'trans_girl', 'trans_other', 'cis'], p=p_gender, size=len(df))

In [22]:
# now distinguish cis based on reconstructed sex_id
df.gender = np.where(df.gender == 'cis',
                     df.sex_id.map({1:'cis_boy', 2:'cis_girl'}),
                     df.gender  # ~50% of trans_boys have reported_sex_2020 female, etc
                    )

In [23]:
df['trans'] = df.gender.isin(['trans_boy', 'trans_girl', 'trans_other'])

In [24]:
np.round(100 * df.gender.value_counts(normalize=True), 2)

cis_boy        50.98
cis_girl       48.49
trans_girl      0.23
trans_boy       0.18
trans_other     0.12
Name: gender, dtype: float64

In [25]:
np.round(100 * df.gender.value_counts(normalize=True).filter(like='trans').sum(), 2)

0.53

In [26]:
np.round(100 * df.trans.mean(), 2)

0.53

In [27]:
df[df.trans].groupby('gender').reported_sex_2020.value_counts(normalize=True).unstack()

reported_sex_2020,1,2
gender,,
trans_boy,0.509158,0.490842
trans_girl,0.512911,0.487089
trans_other,0.525170,0.474830


In [28]:
def gender_to_sex_2010(gender, reported_sex_2020):
    # start with values reported in 2020
    sex = reported_sex_2020.copy()
    
    # update the trans_boy and trans_girl entries to be sex assigned at birth
    sex = np.where((gender == 'trans_boy'),
                   2,
                   sex
                  )
    sex = np.where((gender == 'trans_girl'),
                   1,
                   sex
                  )
    return sex


df['reported_sex_2010'] = gender_to_sex_2010(df.gender, df.reported_sex_2020)

In [29]:
np.mean(df.reported_sex_2010 != df.reported_sex_2020)

0.0020271727553298857

# Values for results section

In [30]:
# Our synthetic population matched the age, sex, race/ethnicity, and geography of Texas
# on census day April 1, 2010, with
# X children ages 0-7 in Y household on census day 2010

n_kids = len(df)  # number of children
f'{n_kids:,.0f}'

'3,009,117'

In [31]:
# back of envelope scenario 1
3_009_117 * (0.18 + 0.23)/100 * 0.5 

6168.689850000001

In [32]:
# back of envelope scenario 2
3_009_117 * 0.002 * 0.23 # * fraction unique

1384.1938200000002

In [33]:
n_households = df.hh_id_2020.nunique()  # number of households
f'{n_households:,.0f}'

'2,354,144'

In [34]:
# number of household that were in same place in 2010 and 2020 census
n_stayed = (df.hh_id_2010 == df.hh_id_2020).sum()
f'{n_stayed:,.0f}'

'691,578'

In [35]:
# number of trans kids that were in same place in 2010 and 2020 census

n_trans_stayed = (df.trans & (df.hh_id_2010 == df.hh_id_2020)).sum()
f'{n_trans_stayed:,.0f}'

'3,625'

In [36]:
# number of trans families identified if full census data with names and dob was released

n_hh_w_sex_different = df[df.reported_sex_2010 != df.reported_sex_2020].hh_id_2010.nunique()
f'{n_hh_w_sex_different:,.0f}'

'6,090'

In [37]:
# number of trans kids identified if full census data with names and dob was released

n_kids_w_sex_different = len(df[df.reported_sex_2010 != df.reported_sex_2020])
f'{n_kids_w_sex_different:,.0f}'

'6,100'

In [38]:
# number of trans kids total

n_trans_kids = sum(df.trans)
f'{n_trans_kids:,.0f}'

'15,951'

In [39]:
np.round(100 * n_kids_w_sex_different/n_trans_kids)

38.0

In [40]:
del df['geoid']
df

,state,county,tract,block,sex_id,relationship,hispanic,racaian,racasn,racblk,...,age_2020,age_2010,hh_id_2020,hh_id_2010,geoid_2020,geoid_2010,reported_sex_2020,gender,trans,reported_sex_2010
2,48,61,980100,1000,1,25,1,0,0,0,...,15,5,480619801001000-1,483090037102003-1,480619801001000,483090037102003,1,cis_boy,False,1
6,48,61,980100,1000,2,25,1,0,0,0,...,10,0,480619801001000-2,484790012011000-16,480619801001000,484790012011000,2,cis_girl,False,2
7,48,61,980100,1000,2,25,1,0,0,0,...,15,5,480619801001000-2,484790012011000-16,480619801001000,484790012011000,2,cis_girl,False,2
16,48,441,11600,1000,1,20,0,0,0,0,...,13,3,484410116001000-52,484410116001000-52,484410116001000,484410116001000,1,cis_boy,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25145326,48,113,7916,2001,2,22,0,0,1,0,...,17,7,481130079162001-743,483396940011002-29,481130079162001,483396940011002,2,cis_girl,False,2
25145453,48,113,7916,1002,2,28,1,0,0,0,...,14,4,481130079161002-738,484910215152000-16,481130079161002,484910215152000,2,cis_girl,False,2
25145494,48,113,7916,2001,1,25,1,0,0,0,...,10,0,481130079162001-788,484910201171011-2,481130079162001,484910201171011,1,cis_boy,False,1
25145495,48,113,7916,2001,1,36,1,0,0,0,...,10,0,481130079162001-791,481410041031009-356,481130079162001,481410041031009,1,cis_boy,False,1


In [41]:
# without reidentification to link on, there is still a risk of identifying a block with a trans kid
# by finding blocks where there was a single kids of a given age in 2010 and that age+10 in 2020 and
# different reported sex
df['reported_male_2010'] = (df.reported_sex_2010 == 1)  # column for easily calculating percent male in each strata
df['geoid'] = df.geoid_2010
df['age'] = df.age_2010 + 10 # add ten to make merge easier

g = df.groupby(['geoid', 'age',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_a = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_a['pct_male'] = 100*g.reported_male_2010.mean()
df_a['pct_trans'] = 100*g.trans.mean()
df_a

n_simulants  \
geoid           age hispanic racwht racblk racasn racaian racnhpi racsor                
480019501001000 16  0        0      1      0      0       0       0                 1   
                17  0        1      0      0      0       0       0                 1   
480019501001001 11  0        0      1      0      0       0       0                 1   
                13  0        1      0      0      0       0       0                 1   
...                                                                               ...   
485079503025024 14  1        1      0      0      0       0       0                 1   
                15  1        1      0      0      0       0       1                 1   
                16  1        1      0      0      0       0       0                 1   
                17  1        1      0      0      0       0       0                 1   

                                                                          pct_male  \
geoid           age hispanic racwht racblk racasn racaian racnhpi racsor             
480019501001000 16  0        0      1      0      0       0       0            0.0   
                17  0        1      0      0      0       0       0          100.0   
480019501001001 11  0        0      1      0      0       0       0          100.0   
                13  0        1      0      0      0       0       0          100.0   
...                                                                            ...   
485079503025024 14  1        1      0      0      0       0       0          100.0   
                15  1        1      0      0      0       0       1            0.0   
                16  1        1      0      0      0       0       0            0.0   
                17  1        1      0      0      0       0       0          100.0   

                                                                          pct_trans  
geoid           age hispanic racwht racblk racasn racaian racnhpi racsor             
480019501001000 16  0        0      1      0      0       0       0             0.0  
                17  0        1      0      0      0       0       0             0.0  
480019501001001 11  0        0      1      0      0       0       0             0.0  
                13  0        1      0      0      0       0       0             0.0  
...                                                                             ...  
485079503025024 14  1        1      0      0      0       0       0             0.0  
                15  1        1      0      0      0       0       1             0.0  
                16  1        1      0      0      0       0       0             0.0  
                17  1        1      0      0      0       0       0             0.0  

[2211007 rows x 3 columns]

In [42]:
# add a column for age group, as aggregated in P12* tables in SF1
# --- this provides a proxy for how there will be less population
# uniques when aggregation error is not addressed with some sort of
# reconstruction
from linked_census_disclosure.data import p12_age_group_map

df['age_group'] = df.age.map(p12_age_group_map)
g = df.groupby(['geoid', 'age_group',  # notice that I have switched from age to age group here
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_a_2 = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_a_2['pct_male'] = 100*g.reported_male_2010.mean()
df_a_2['pct_trans'] = 100*g.trans.mean()
df_a_2

n_simulants  \
geoid           age_group hispanic racwht racblk racasn racaian racnhpi racsor                
480019501001000 3         0        0      1      0      0       0       0                 1   
                                   1      0      0      0       0       0                 1   
480019501001001 2         0        0      1      0      0       0       0                 1   
                                   1      0      0      0       0       0                 1   
...                                                                                     ...   
485079503025024 2         1        0      0      0      0       0       1                 1   
                                   1      0      0      0       0       0                 3   
                3         1        1      0      0      0       0       0                 2   
                                                                        1                 1   

                                                                                  pct_male  \
geoid           age_group hispanic racwht racblk racasn racaian racnhpi racsor               
480019501001000 3         0        0      1      0      0       0       0         0.000000   
                                   1      0      0      0       0       0       100.000000   
480019501001001 2         0        0      1      0      0       0       0       100.000000   
                                   1      0      0      0       0       0       100.000000   
...                                                                                    ...   
485079503025024 2         1        0      0      0      0       0       1       100.000000   
                                   1      0      0      0       0       0        33.333333   
                3         1        1      0      0      0       0       0        50.000000   
                                                                        1         0.000000   

                                                                                pct_trans  
geoid           age_group hispanic racwht racblk racasn racaian racnhpi racsor             
480019501001000 3         0        0      1      0      0       0       0             0.0  
                                   1      0      0      0       0       0             0.0  
480019501001001 2         0        0      1      0      0       0       0             0.0  
                                   1      0      0      0       0       0             0.0  
...                                                                                   ...  
485079503025024 2         1        0      0      0      0       0       1             0.0  
                                   1      0      0      0       0       0             0.0  
                3         1        1      0      0      0       0       0             0.0  
                                                                        1             0.0  

[1414951 rows x 3 columns]

In [43]:
# the rows with n_simulants == 1 are all strata with a single simulant in 2010
# (I used age_2020 in the index to make it easier to merge them with the df_b I will construct next)
n_unique_2010 = sum(df_a.n_simulants == 1)
n_unique_2010

1722991

In [44]:
df['reported_male_2020'] = (df.reported_sex_2020 == 1)
df['geoid'] = df.geoid_2020
df['age'] = df.age_2020

df['discordant_sex'] = (df.reported_sex_2010 != df.reported_sex_2020)
g = df.groupby(['geoid', 'age',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_b = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_b['pct_male'] = 100*g.reported_male_2020.mean()
df_b['pct_trans'] = 100*g.trans.mean()
df_b['pct_discordant_sex'] = 100*g.discordant_sex.mean()
n_unique_2020 = sum(df_b.n_simulants == 1)
n_unique_2020

1413801

In [45]:
df_ab = df_a[df_a.n_simulants==1].copy()
df_ab.columns = ['n_simulants_2010', 'pct_male_2010', 'pct_trans_2010']

df_ab['n_simulants_2020'] = df_b[df_b.n_simulants==1].n_simulants
df_ab['pct_male_2020'] = df_b[df_b.n_simulants==1].pct_male
df_ab['pct_trans_2020'] = df_b[df_b.n_simulants==1].pct_trans
df_ab['pct_discordant_sex'] = df_b[df_b.n_simulants==1].pct_discordant_sex
df_ab = df_ab.dropna()

In [46]:
df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020]

,,,,,,,,,n_simulants_2010,pct_male_2010,pct_trans_2010,n_simulants_2020,pct_male_2020,pct_trans_2020,pct_discordant_sex
geoid,age,hispanic,racwht,racblk,racasn,racaian,racnhpi,racsor,,,,,,,
480019501001022,15,0,0,1,0,0,0,0,1,0.0,0.0,1.0,100.0,0.0,0.0
480019501002073,16,0,1,0,0,0,0,0,1,0.0,0.0,1.0,100.0,0.0,0.0
480019501002086,11,0,1,0,0,0,0,0,1,100.0,0.0,1.0,0.0,0.0,0.0
480019501002112,11,0,1,0,0,0,0,0,1,0.0,0.0,1.0,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485079503025006,12,1,1,0,0,0,0,0,1,100.0,0.0,1.0,0.0,0.0,0.0
485079503025008,13,1,1,0,0,0,0,0,1,0.0,0.0,1.0,100.0,0.0,0.0
485079503025017,17,1,1,0,0,0,0,0,1,100.0,0.0,1.0,0.0,0.0,0.0
485079503025023,16,1,1,0,0,0,0,0,1,100.0,0.0,1.0,0.0,0.0,0.0


In [47]:
df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020].pct_trans_2020.value_counts()

0.0      69021
100.0      938
Name: pct_trans_2020, dtype: int64

In [48]:
df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020].pct_discordant_sex.value_counts()

0.0      69268
100.0      691
Name: pct_discordant_sex, dtype: int64

In [49]:
df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020].pct_discordant_sex.mean()

0.9877213796652325

In [50]:
n_scenario_2_no_aggregation_error = sum(
    df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020].pct_discordant_sex == 100
    )
N_scenario_2_no_aggregation_error = sum(df_ab.pct_male_2010 != df_ab.pct_male_2020)

In [51]:
n_scenario_2_no_aggregation_error, N_scenario_2_no_aggregation_error

(691, 69959)

In [52]:
((df_ab.pct_male_2010 != df_ab.pct_male_2020)*1.0).describe()

count    407733.000000
mean          0.171580
std           0.377016
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
dtype: float64

In [53]:
# add a column for age group, as aggregated in P12* tables in SF1
# --- this provides a proxy for how there will be less population
# uniques when aggregation error is not addressed with some sort of
# reconstruction
df['age_group'] = df.age.map(p12_age_group_map)

g = df.groupby(['geoid', 'age_group',  # notice that I have switched from age to age group here
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_b_2 = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_b_2['pct_male'] = 100*g.reported_male_2020.mean()
df_b_2['pct_trans'] = 100*g.trans.mean()
df_b_2['pct_discordant_sex'] = 100*g.discordant_sex.mean()
n_unique_2020_2 = sum(df_b_2.n_simulants == 1)
n_unique_2020_2

573415

In [54]:
df_ab_2 = df_a_2[df_a_2.n_simulants==1].copy()
df_ab_2.columns = ['n_simulants_2010', 'pct_male_2010', 'pct_trans_2010']

df_ab_2['n_simulants_2020'] = df_b_2[df_b_2.n_simulants==1].n_simulants
df_ab_2['pct_male_2020'] = df_b_2[df_b_2.n_simulants==1].pct_male
df_ab_2['pct_trans_2020'] = df_b_2[df_b_2.n_simulants==1].pct_trans
df_ab_2['pct_discordant_sex'] = df_b_2[df_b_2.n_simulants==1].pct_discordant_sex
# df_ab_2 = df_ab_2.dropna()

n_scenario_2_max_aggregation_error = sum(
    df_ab_2[df_ab_2.pct_male_2010 != df_ab_2.pct_male_2020].pct_discordant_sex == 100
    )
N_scenario_2_max_aggregation_error = sum(df_ab_2.pct_male_2010 != df_ab_2.pct_male_2020)

n_scenario_2_max_aggregation_error, N_scenario_2_max_aggregation_error

(251, 693856)

In [55]:
# 691 identified is an over-estimate and 251 is an underestimate
# to find out where in between, I will take the 691 that would be
# identified by perfect reconstruction, and see how many are
# population uniques after tabulation and reconstruction of their
# census block adds aggregation error

identified_with_perfect_reconstruction = df_ab[(df_ab.pct_male_2010 != df_ab.pct_male_2020)
                                              & (df_ab.pct_discordant_sex == 100)].index
assert len(identified_with_perfect_reconstruction)  == 691

df_ab.loc[identified_with_perfect_reconstruction]

,,,,,,,,,n_simulants_2010,pct_male_2010,pct_trans_2010,n_simulants_2020,pct_male_2020,pct_trans_2020,pct_discordant_sex
geoid,age,hispanic,racwht,racblk,racasn,racaian,racnhpi,racsor,,,,,,,
480019508002036,12,0,1,0,0,0,0,0,1,100.0,100.0,1.0,0.0,100.0,100.0
480019509013021,17,0,1,0,0,0,0,0,1,100.0,100.0,1.0,0.0,100.0,100.0
480050006001026,12,0,0,1,0,0,0,0,1,0.0,0.0,1.0,100.0,100.0,100.0
480050006003005,15,1,0,0,0,0,0,1,1,100.0,100.0,1.0,0.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484999501004064,11,0,1,0,0,0,0,0,1,0.0,0.0,1.0,100.0,100.0,100.0
485019501001165,15,1,0,0,0,0,0,1,1,100.0,0.0,1.0,0.0,100.0,100.0
485039504011014,12,0,1,0,0,0,0,0,1,0.0,100.0,1.0,100.0,100.0,100.0
485079503021004,15,1,0,0,0,0,0,1,1,100.0,100.0,1.0,0.0,100.0,100.0


In [56]:
len(identified_with_perfect_reconstruction)

691

In [57]:
def geoid_to_codes(geoid):
    """convert geoid to state, county, tract, block"""
    geoid = str(geoid)
#     print(geoid)
    
    state = int(geoid[0:2])
    county = int(geoid[2:5])
    tract = int(geoid[5:11])
    block = int(geoid[-4:])
    
    del geoid  # HACK to make locals 
    return locals()
geoid_to_codes(identified_with_perfect_reconstruction[0][0])

{'state': 48, 'county': 1, 'tract': 950800, 'block': 2036}

In [58]:
import linked_census_disclosure.model
from tqdm import tqdm

In [59]:
n_reconstructed_match = 0
total_iterations = len(identified_with_perfect_reconstruction)

for i in tqdm(identified_with_perfect_reconstruction, total=total_iterations, desc="Processing"):
    s_i = df_ab.loc[i]
    geoid, age, hispanic, racwht, racblk, racasn, racaian, racnhpi, racsor = i

    n_times_matched_i_uniquely = 0
    for y, df_block in enumerate([df[df.geoid_2010 == geoid], df[df.geoid_2020 == geoid]]):
        df_block = df_block.copy()
        if y == 0:
            df_block.loc[:, 'sex_id'] = df_block.reported_sex_2010
            sex = 1-s_i.pct_male_2010/100
        else:
            df_block.loc[:, 'sex_id'] = df_block.reported_sex_2020
            sex = 1-s_i.pct_male_2020/100
        
        df_w_aggregation_error = linked_census_disclosure.model.aggregate_and_reconstruct_block(
            df_block,
            **geoid_to_codes(geoid))


        df_reconstructed_match = df_w_aggregation_error.query(
            'age == @age and hispanic == @hispanic and '
            'racwht == @racwht and racblk == @racblk and racaian == @racaian '
            'and racasn==@racasn and racnhpi==@racnhpi and racsor == @racsor '
            'and sex==@sex'
        )
        if df_reconstructed_match.n.sum() == 1:
            n_times_matched_i_uniquely += 1

    # also confirm that reconstructed sex differs
    if (n_times_matched_i_uniquely == 2):
            n_reconstructed_match += 1

Processing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 691/691 [41:31<00:00,  3.61s/it]


In [60]:
df_w_aggregation_error

,age,sex,race,eth,n,sex_id,racwht,racblk,racaian,racasn,racnhpi,racsor,hispanic
0,11,1,5,1,1.0,2,0,0,0,0,0,1,1
1,15,1,5,1,1.0,2,0,0,0,0,0,1,1


In [61]:
df_reconstructed_match

,age,sex,race,eth,n,sex_id,racwht,racblk,racaian,racasn,racnhpi,racsor,hispanic
1,15,1,5,1,1.0,2,0,0,0,0,0,1,1


In [62]:
n_scenario_2_address_aggregation_error = n_reconstructed_match
n_scenario_2_address_aggregation_error # at most 691, and at least 251 --- I'm guessing 500

307

In [63]:
# I better check that again
n_scenario_2 = n_scenario_2_address_aggregation_error

# next a version with household swapping to protect against disclosure
# I hypothesize that it is going to be just 10% lower

In [64]:
p_swap = 0.05

In [65]:
locations_2010 = df.hh_id_2010.unique()

random_location = np.random.choice(all_locations, size=len(locations_2010),  # induces distribution on geoid that is proportional to number of households
                                   replace=True)

reported_locations_2010 = np.where(np.random.uniform(size=len(locations_2010)) < p_swap,
                                   random_location,
                                   locations_2010,
                                  )

s_reported_location_2010 = pd.Series(reported_locations_2010,
                                     index=locations_2010)

df['reported_hh_id_2010'] = df.hh_id_2010.map(s_reported_location_2010)

In [66]:
locations_2020 = df.hh_id_2020.unique()

random_location = np.random.choice(all_locations, size=len(locations_2020),
                                   replace=True)

reported_locations_2020 = np.where(np.random.uniform(size=len(locations_2020)) < p_swap,
                                   random_location,
                                   locations_2020,
                                  )

s_reported_location_2020 = pd.Series(reported_locations_2020,
                                     index=locations_2020)

df['reported_hh_id_2020'] = df.hh_id_2020.map(s_reported_location_2020)

In [67]:
# number of trans families identified by age, geoid from reconstruction without noise

df['reported_geoid_2010'] = df.reported_hh_id_2010.map(lambda x: int(x.split('-')[0]))
df['reported_geoid_2020'] = df.reported_hh_id_2020.map(lambda x: int(x.split('-')[0]))

In [68]:
# without reidentification to link on, there is still a risk of identifying a block with a trans kid
# by finding blocks where there was a single kids of a given age in 2010 and that age+10 in 2020 and
# different reported sex
df['reported_male_2010'] = (df.reported_sex_2010 == 1)
df['geoid'] = df.reported_geoid_2010
df['age'] = df.age_2010 + 10 # add ten to make merge easier

g = df.groupby(['geoid', 'age',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_a = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_a['pct_male'] = 100*g.reported_male_2010.mean()
df_a['pct_trans'] = 100*g.trans.mean()
# df_a

In [69]:
df['reported_male_2020'] = (df.reported_sex_2020 == 1)
df['geoid'] = df.reported_geoid_2020
df['age'] = df.age_2020

df['discordant_sex'] = (df.reported_sex_2010 != df.reported_sex_2020)
g = df.groupby(['geoid', 'age',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_b = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_b['pct_male'] = 100*g.reported_male_2020.mean()
df_b['pct_trans'] = 100*g.trans.mean()
df_b['pct_discordant_sex'] = 100*g.discordant_sex.mean()
n_unique_2020 = sum(df_b.n_simulants == 1)
n_unique_2020

1454146

In [70]:
df_ab = df_a[df_a.n_simulants==1].copy()
df_ab.columns = ['n_simulants_2010', 'pct_male_2010', 'pct_trans_2010']

df_ab['n_simulants_2020'] = df_b[df_b.n_simulants==1].n_simulants
df_ab['pct_male_2020'] = df_b[df_b.n_simulants==1].pct_male
df_ab['pct_trans_2020'] = df_b[df_b.n_simulants==1].pct_trans
df_ab['pct_discordant_sex'] = df_b[df_b.n_simulants==1].pct_discordant_sex

df_ab = df_ab.dropna()

In [71]:
n_scenario_3_no_aggregation_error = sum(
    df_ab[df_ab.pct_male_2010 != df_ab.pct_male_2020].pct_discordant_sex == 100
    )
N_scenario_3_no_aggregation_error = sum(df_ab.pct_male_2010 != df_ab.pct_male_2020)
n_scenario_3_no_aggregation_error, N_scenario_3_no_aggregation_error-n_scenario_3_no_aggregation_error

(664, 77554)

## Again, but with max aggregation error

CHECK CODE BELOW, MIGHT HAVE BUG

In [72]:
df['age'] = df.age_2010 + 10 # add ten to make merge easier
# add a column for age group, as aggregated in P12* tables in SF1
# --- this provides a proxy for how there will be less population
# uniques when aggregation error is not addressed with some sort of
# reconstruction
df['age_group'] = df.age.map(p12_age_group_map)

g = df.groupby(['geoid', 'age_group',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_a_2 = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_a_2['pct_male'] = 100*g.reported_male_2010.mean()
df_a_2['pct_trans'] = 100*g.trans.mean()



df['age'] = df.age_2020
# add a column for age group, as aggregated in P12* tables in SF1
# --- this provides a proxy for how there will be less population
# uniques when aggregation error is not addressed with some sort of
# reconstruction
df['age_group'] = df.age.map(p12_age_group_map)

df['discordant_sex'] = (df.reported_sex_2010 != df.reported_sex_2020)
g = df.groupby(['geoid', 'age_group',
                'hispanic', 'racwht', 'racblk',
                'racasn', 'racaian', 'racnhpi', 'racsor'])

df_b_2 = pd.DataFrame({'n_simulants': g.pweight.sum()})
df_b_2['pct_male'] = 100*g.reported_male_2020.mean()
df_b_2['pct_trans'] = 100*g.trans.mean()
df_b_2['pct_discordant_sex'] = 100*g.discordant_sex.mean()

df_ab_2 = df_a_2[df_a_2.n_simulants==1].copy()
df_ab_2.columns = ['n_simulants_2010', 'pct_male_2010', 'pct_trans_2010']

df_ab_2['n_simulants_2020'] = df_b_2[df_b_2.n_simulants==1].n_simulants
df_ab_2['pct_male_2020'] = df_b_2[df_b_2.n_simulants==1].pct_male
df_ab_2['pct_trans_2020'] = df_b_2[df_b_2.n_simulants==1].pct_trans
df_ab_2['pct_discordant_sex'] = df_b_2[df_b_2.n_simulants==1].pct_discordant_sex

df_ab_2 = df_ab_2.dropna()


n_scenario_3_max_aggregation_error = sum(
    df_ab_2[df_ab_2.pct_male_2010 != df_ab_2.pct_male_2020].pct_discordant_sex == 100
    )
N_scenario_3_max_aggregation_error = sum(df_ab_2.pct_male_2010 != df_ab_2.pct_male_2020)
n_scenario_3_max_aggregation_error, N_scenario_3_max_aggregation_error-n_scenario_3_max_aggregation_error # expect 250

(1223, 0)

# Figure out where it is in between

In [73]:
identified_with_perfect_reconstruction = df_ab[(df_ab.pct_male_2010 != df_ab.pct_male_2020)
                                              & (df_ab.pct_discordant_sex == 100)].index
assert len(identified_with_perfect_reconstruction)  == 664

df_ab.loc[identified_with_perfect_reconstruction]

,,,,,,,,,n_simulants_2010,pct_male_2010,pct_trans_2010,n_simulants_2020,pct_male_2020,pct_trans_2020,pct_discordant_sex
geoid,age,hispanic,racwht,racblk,racasn,racaian,racnhpi,racsor,,,,,,,
480019508002036,12,0,1,0,0,0,0,0,1,100.0,100.0,1.0,0.0,100.0,100.0
480019509013021,17,0,1,0,0,0,0,0,1,100.0,100.0,1.0,0.0,100.0,100.0
480050006001026,12,0,0,1,0,0,0,0,1,0.0,0.0,1.0,100.0,100.0,100.0
480050006003005,15,1,0,0,0,0,0,1,1,100.0,100.0,1.0,0.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484999501004064,11,0,1,0,0,0,0,0,1,0.0,0.0,1.0,100.0,100.0,100.0
485019501001165,15,1,0,0,0,0,0,1,1,100.0,0.0,1.0,0.0,100.0,100.0
485039504011014,12,0,1,0,0,0,0,0,1,0.0,100.0,1.0,100.0,100.0,100.0
485079503021004,15,1,0,0,0,0,0,1,1,100.0,100.0,1.0,0.0,100.0,100.0


# TODO: don't repeat yourself, the following is same code as in cell 59


In [74]:
n_reconstructed_match = 0
total_iterations = len(identified_with_perfect_reconstruction)

for i in tqdm(identified_with_perfect_reconstruction, total=total_iterations, desc="Processing"):
    s_i = df_ab.loc[i]
    geoid, age, hispanic, racwht, racblk, racasn, racaian, racnhpi, racsor = i

    n_times_matched_i_uniquely = 0
    for y, df_block in enumerate([df[df.geoid_2010 == geoid], df[df.geoid_2020 == geoid]]):
        df_block = df_block.copy()
        if y == 0:
            df_block.loc[:, 'sex_id'] = df_block.reported_sex_2010
            sex = 1-s_i.pct_male_2010/100
        else:
            df_block.loc[:, 'sex_id'] = df_block.reported_sex_2020
            sex = 1-s_i.pct_male_2020/100
        
        df_w_aggregation_error = linked_census_disclosure.model.aggregate_and_reconstruct_block(
            df_block,
            **geoid_to_codes(geoid))


        df_reconstructed_match = df_w_aggregation_error.query(
            'age == @age and hispanic == @hispanic and '
            'racwht == @racwht and racblk == @racblk and racaian == @racaian '
            'and racasn==@racasn and racnhpi==@racnhpi and racsor == @racsor '
            'and sex==@sex'
        )
        if df_reconstructed_match.n.sum() == 1:
            n_times_matched_i_uniquely += 1

    # also confirm that reconstructed sex differs
    if (n_times_matched_i_uniquely == 2):
            n_reconstructed_match += 1

Processing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 664/664 [26:53<00:00,  2.43s/it]


In [75]:
n_scenario_3_address_aggregation_error = n_reconstructed_match
n_scenario_3_address_aggregation_error # at most 644, and at least 250 --- I'm guessing 400, since I expect it to be a little lower than scenario 2

283

In [76]:
n_scenario_3_no_aggregation_error / N_scenario_3_no_aggregation_error

0.008489094581809813

In [77]:
n_scenario_3 = n_scenario_3_address_aggregation_error

In [78]:
n_scenario_2, N_scenario_2_no_aggregation_error-n_scenario_2

(307, 69652)

In [79]:
n_scenario_2 / N_scenario_2_no_aggregation_error

0.004388284566674766

In [80]:
100 * (n_scenario_2 - n_scenario_3) / n_scenario_2  # pct decrease from scenario 2 to 3

7.817589576547231